In [108]:

import os
from TTS.utils.audio import AudioProcessor
from TTS.config import load_config
from TTS.tts.models import setup_model as setup_tts_model
import torch
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline



BASE_DIR = '/home/perry/PycharmProjects/TTS/recipes/ljspeech/prune/tacotron2_nomask/'
ZERO_DIR = BASE_DIR + 'baseline/'
ZERO_FILE = ZERO_DIR + "checkpoint_100000.pth.tar"
CONFIG_PATH = ZERO_DIR + "config.json"
DATA_DIR = "/home/perry/PycharmProjects/TTS/recipes/ljspeech/LJSpeech-1.1/"

In [109]:
plt.style.use('dark_background')

In [110]:
C = load_config(CONFIG_PATH)
ap = AudioProcessor(**C.audio)
model = setup_tts_model(C)
model.load_checkpoint(C, ZERO_FILE, eval=True)
model = model.cuda()

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20.0
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Using model: tacotron2
 > Model's reduction rate `r` is set to: 1


In [111]:
for name, param in model.named_parameters():
    print(name, param.shape)

embedding.weight torch.Size([200, 512])
encoder.convolutions.0.convolution1d.weight torch.Size([512, 512, 5])
encoder.convolutions.0.convolution1d.bias torch.Size([512])
encoder.convolutions.0.batch_normalization.weight torch.Size([512])
encoder.convolutions.0.batch_normalization.bias torch.Size([512])
encoder.convolutions.1.convolution1d.weight torch.Size([512, 512, 5])
encoder.convolutions.1.convolution1d.bias torch.Size([512])
encoder.convolutions.1.batch_normalization.weight torch.Size([512])
encoder.convolutions.1.batch_normalization.bias torch.Size([512])
encoder.convolutions.2.convolution1d.weight torch.Size([512, 512, 5])
encoder.convolutions.2.convolution1d.bias torch.Size([512])
encoder.convolutions.2.batch_normalization.weight torch.Size([512])
encoder.convolutions.2.batch_normalization.bias torch.Size([512])
encoder.lstm.weight_ih_l0 torch.Size([1024, 512])
encoder.lstm.weight_hh_l0 torch.Size([1024, 256])
encoder.lstm.bias_ih_l0 torch.Size([1024])
encoder.lstm.bias_hh_l0 t

In [112]:
for n, p in model.named_parameters():
    print(n, p.numel(), p.count_nonzero().item())

embedding.weight 102400 101888
encoder.convolutions.0.convolution1d.weight 1310720 1310720
encoder.convolutions.0.convolution1d.bias 512 512
encoder.convolutions.0.batch_normalization.weight 512 512
encoder.convolutions.0.batch_normalization.bias 512 512
encoder.convolutions.1.convolution1d.weight 1310720 1310720
encoder.convolutions.1.convolution1d.bias 512 512
encoder.convolutions.1.batch_normalization.weight 512 512
encoder.convolutions.1.batch_normalization.bias 512 512
encoder.convolutions.2.convolution1d.weight 1310720 1310720
encoder.convolutions.2.convolution1d.bias 512 512
encoder.convolutions.2.batch_normalization.weight 512 512
encoder.convolutions.2.batch_normalization.bias 512 512
encoder.lstm.weight_ih_l0 524288 524288
encoder.lstm.weight_hh_l0 262144 262144
encoder.lstm.bias_ih_l0 1024 1024
encoder.lstm.bias_hh_l0 1024 1024
encoder.lstm.weight_ih_l0_reverse 524288 524288
encoder.lstm.weight_hh_l0_reverse 262144 262144
encoder.lstm.bias_ih_l0_reverse 1024 1024
encoder.lst

In [113]:
from TTS.utils.generic_utils import get_module_by_name
param_prune_dim = {'embedding.weight': 1,
                   'encoder.convolutions.0.convolution1d.weight': 0,
                   'encoder.convolutions.1.convolution1d.weight': 0,
                   'encoder.convolutions.2.convolution1d.weight': 0,
                   'encoder.lstm.weight_ih_l0': 0,
                   'encoder.lstm.weight_hh_l0': 0,
                   'encoder.lstm.weight_ih_l0_reverse': 0,
                   'encoder.lstm.weight_hh_l0_reverse': 0,
                   'decoder.prenet.linear_layers.0.linear_layer.weight': 0,
                   'decoder.attention_rnn.weight_ih': 0,
                   'decoder.attention_rnn.weight_hh': 0,
                   'decoder.attention.query_layer.weight': 0,
                   'decoder.attention.key_layer.weight': 0,
                   'decoder.decoder_rnn.weight_ih': 0,
                   'decoder.decoder_rnn.weight_hh': 0,
                   'postnet.convolutions.0.convolution1d.weight': 0,
                   'postnet.convolutions.1.convolution1d.weight': 0,
                   'postnet.convolutions.2.convolution1d.weight': 0,
                   'postnet.convolutions.3.convolution1d.weight': 0,
                   'coarse_decoder.prenet.linear_layers.0.linear_layer.weight': 0,
                   'coarse_decoder.prenet.linear_layers.1.linear_layer.weight': 0,
                   'coarse_decoder.attention_rnn.weight_ih': 0,
                   'coarse_decoder.attention_rnn.weight_hh': 0,
                   'coarse_decoder.attention.query_layer.weight': 0,
                   'coarse_decoder.attention.key_layer.weight': 0,
                   'coarse_decoder.decoder_rnn.weight_ih': 0,
                   'coarse_decoder.decoder_rnn.weight_hh': 0,
                    }

In [114]:
UMP_DIR = BASE_DIR + 'ump/'
UMP_10_FILE = UMP_DIR + 'sparsity_10_stop/checkpoint_100000.pth.tar'
config = load_config(CONFIG_PATH)
ump_model = setup_tts_model(config)
ump_model.load_checkpoint(config, UMP_10_FILE, eval=True)
ump_model = ump_model.cuda()

 > Using model: tacotron2
 > Model's reduction rate `r` is set to: 1


In [115]:
for n, p in ump_model.named_parameters():
    print(n, p.size())

embedding.weight torch.Size([200, 512])
encoder.convolutions.0.convolution1d.weight torch.Size([512, 512, 5])
encoder.convolutions.0.convolution1d.bias torch.Size([512])
encoder.convolutions.0.batch_normalization.weight torch.Size([512])
encoder.convolutions.0.batch_normalization.bias torch.Size([512])
encoder.convolutions.1.convolution1d.weight torch.Size([512, 512, 5])
encoder.convolutions.1.convolution1d.bias torch.Size([512])
encoder.convolutions.1.batch_normalization.weight torch.Size([512])
encoder.convolutions.1.batch_normalization.bias torch.Size([512])
encoder.convolutions.2.convolution1d.weight torch.Size([512, 512, 5])
encoder.convolutions.2.convolution1d.bias torch.Size([512])
encoder.convolutions.2.batch_normalization.weight torch.Size([512])
encoder.convolutions.2.batch_normalization.bias torch.Size([512])
encoder.lstm.weight_ih_l0 torch.Size([1024, 512])
encoder.lstm.weight_hh_l0 torch.Size([1024, 256])
encoder.lstm.bias_ih_l0 torch.Size([1024])
encoder.lstm.bias_hh_l0 t

In [116]:
ump_params = {n: (p.numel(), p.count_nonzero().item()) for n, p in ump_model.named_parameters()}

In [117]:
ump_params

{'embedding.weight': (102400, 101616),
 'encoder.convolutions.0.convolution1d.weight': (1310720, 1110363),
 'encoder.convolutions.0.convolution1d.bias': (512, 417),
 'encoder.convolutions.0.batch_normalization.weight': (512, 512),
 'encoder.convolutions.0.batch_normalization.bias': (512, 512),
 'encoder.convolutions.1.convolution1d.weight': (1310720, 1098397),
 'encoder.convolutions.1.convolution1d.bias': (512, 437),
 'encoder.convolutions.1.batch_normalization.weight': (512, 512),
 'encoder.convolutions.1.batch_normalization.bias': (512, 512),
 'encoder.convolutions.2.convolution1d.weight': (1310720, 1091901),
 'encoder.convolutions.2.convolution1d.bias': (512, 424),
 'encoder.convolutions.2.batch_normalization.weight': (512, 512),
 'encoder.convolutions.2.batch_normalization.bias': (512, 512),
 'encoder.lstm.weight_ih_l0': (524288, 492748),
 'encoder.lstm.weight_hh_l0': (262144, 248566),
 'encoder.lstm.bias_ih_l0': (1024, 966),
 'encoder.lstm.bias_hh_l0': (1024, 987),
 'encoder.lstm.

In [118]:
units_to_prune = {}
for name in param_prune_dim:
    numel, nonzero = ump_params[name]
    units_to_prune[name] = (1 - nonzero / numel) / 10

In [119]:
units_to_prune

{'embedding.weight': 0.0007656250000000031,
 'encoder.convolutions.0.convolution1d.weight': 0.01528602600097656,
 'encoder.convolutions.1.convolution1d.weight': 0.01619895935058594,
 'encoder.convolutions.2.convolution1d.weight': 0.016694564819335934,
 'encoder.lstm.weight_ih_l0': 0.006015777587890625,
 'encoder.lstm.weight_hh_l0': 0.005179595947265625,
 'encoder.lstm.weight_ih_l0_reverse': 0.005892181396484375,
 'encoder.lstm.weight_hh_l0_reverse': 0.005263519287109375,
 'decoder.prenet.linear_layers.0.linear_layer.weight': 0.002578124999999998,
 'decoder.attention_rnn.weight_ih': 0.010407702128092444,
 'decoder.attention_rnn.weight_hh': 0.0088029146194458,
 'decoder.attention.query_layer.weight': 0.010019683837890625,
 'decoder.attention.key_layer.weight': 0.0035900297619047674,
 'decoder.decoder_rnn.weight_ih': 0.009568945566813147,
 'decoder.decoder_rnn.weight_hh': 0.007475376129150391,
 'postnet.convolutions.0.convolution1d.weight': 0.006802246093749997,
 'postnet.convolutions.1.c

In [120]:
from torch.nn.utils import prune
for name, amount in units_to_prune.items():
    m, n = name.rsplit('.', 1)
    module = get_module_by_name(model, m)
    dim = param_prune_dim[name]
    param = getattr(module, n)
    print(f'Pruning {name}: {param.shape[dim]} channels at ratio {amount}')
    if param.shape[dim] * amount > 0.5:
        prune.ln_structured(module, n, amount, 1, dim, importance_scores=None)
        prune.remove(module, n)
        bias_name = n.replace('weight', 'bias')
        if hasattr(module, bias_name):
            bias = getattr(module, bias_name)
            if bias is not None:
                print(f'Pruning {m}.{bias_name}')
                add_axes = list(range(param.dim()))
                add_axes.pop(dim)
                param_sum = param.abs().sum(axis=add_axes)
                bias = bias * (param_sum > 0.0)
                bias = torch.nn.Parameter(bias)
                setattr(module, bias_name, bias)

Pruning embedding.weight: 512 channels at ratio 0.0007656250000000031)
Pruning encoder.convolutions.0.convolution1d.weight: 512 channels at ratio 0.01528602600097656)
Pruning encoder.convolutions.0.convolution1d.bias
Pruning encoder.convolutions.1.convolution1d.weight: 512 channels at ratio 0.01619895935058594)
Pruning encoder.convolutions.1.convolution1d.bias
Pruning encoder.convolutions.2.convolution1d.weight: 512 channels at ratio 0.016694564819335934)
Pruning encoder.convolutions.2.convolution1d.bias
Pruning encoder.lstm.weight_ih_l0: 1024 channels at ratio 0.006015777587890625)
Pruning encoder.lstm.bias_ih_l0
Pruning encoder.lstm.weight_hh_l0: 1024 channels at ratio 0.005179595947265625)
Pruning encoder.lstm.bias_hh_l0
Pruning encoder.lstm.weight_ih_l0_reverse: 1024 channels at ratio 0.005892181396484375)
Pruning encoder.lstm.bias_ih_l0_reverse
Pruning encoder.lstm.weight_hh_l0_reverse: 1024 channels at ratio 0.005263519287109375)
Pruning encoder.lstm.bias_hh_l0_reverse
Pruning de

In [15]:
from TTS.utils.generic_utils import get_param_by_name
param = get_param_by_name(model, 'encoder.convolutions.0.convolution1d.weight')
param.shape

torch.Size([512, 512, 5])

In [36]:
param_sum = param.abs().sum(axis=[1,2])

In [46]:
bias = get_param_by_name(model, 'encoder.convolutions.0.convolution1d.bias')


In [48]:
bias = bias * (param_sum > 0.0)
bias = torch.nn.Parameter(bias)

In [52]:
m, n = 'encoder.convolutions.0.convolution1d.bias'.rsplit('.', 1)
module = get_module_by_name(model, m)
setattr(module, n, bias)

In [55]:
hasattr(module, 'birras')

False

In [45]:
total_removed = 0
for n, b in model.named_buffers():
    if 'mask' in n:
        removed = (b.numel() - b.count_nonzero().item())
        total_removed += removed
        print(n, removed)
print(total_removed)

0


In [121]:
for n, p in model.named_parameters():
    print(n, p.numel(), p.count_nonzero().item())

embedding.weight 102400 101888
encoder.convolutions.0.convolution1d.bias 512 504
encoder.convolutions.0.convolution1d.weight 1310720 1290240
encoder.convolutions.0.batch_normalization.weight 512 512
encoder.convolutions.0.batch_normalization.bias 512 512
encoder.convolutions.1.convolution1d.bias 512 504
encoder.convolutions.1.convolution1d.weight 1310720 1290240
encoder.convolutions.1.batch_normalization.weight 512 512
encoder.convolutions.1.batch_normalization.bias 512 512
encoder.convolutions.2.convolution1d.bias 512 503
encoder.convolutions.2.convolution1d.weight 1310720 1287680
encoder.convolutions.2.batch_normalization.weight 512 512
encoder.convolutions.2.batch_normalization.bias 512 512
encoder.lstm.bias_ih_l0 1024 1018
encoder.lstm.bias_hh_l0 1024 1019
encoder.lstm.bias_ih_l0_reverse 1024 1018
encoder.lstm.bias_hh_l0_reverse 1024 1019
encoder.lstm.weight_ih_l0 524288 521216
encoder.lstm.weight_hh_l0 262144 260864
encoder.lstm.weight_ih_l0_reverse 524288 521216
encoder.lstm.weig

In [122]:
from TTS.utils.generic_utils import count_parameters

In [123]:
total_removed / count_parameters(model)

NameError: name 'total_removed' is not defined

In [125]:
from TTS.utils.io import save_model
output_path = os.path.join(BASE_DIR, 'trim', 'sparsity_5', 'checkpoint_100000.pth.tar')
save_model(config, model,
                   optimizer=None,
                   scheduler=None,
                   scaler=None,
                   current_step=100000,
                   epoch=None,
                   output_path=output_path)